﻿
## 股票数据
你已经了解了时间序列数据，下面学习如何将股价数据放入 Pandas 中。
## 读取数据
数据集可能是各种不同的格式。最常见的格式是 [CSV](https://en.wikipedia.org/wiki/Comma-separated_values)。我们将使用“prices.csv”文件作为示例 csv 文件。

In [ ]:
with open('prices.csv', 'r') as file:
    prices = file.read()
    
print(prices)

数据提供器将提供关于 CSV 中每个字段的信息。此 csv 包含以下字段：ticker、date、open、high、low、close、volume、adj_close、adj_volume（按顺序排列）。第一行包含以下数据：

- ticker:ABC
- date:2017-09-05
- open:163.09
- high:164.24
- low:160.21
- close:162.63
- volume:29417590.0
- adj_close:162.49
- adj_volume:29414672.0

我们将此数据放入 DataFrame 中。我们将使用[`pd.read_csv`](https://pandas.pydata.org/pandas-docs/version/0.21.1/generated/pandas.read_csv.html) 函数完成此操作。此函数使我们能够根据 CSV 数据生成一个 DataFrame。

In [ ]:
import pandas as pd

price_df = pd.read_csv('prices.csv')

price_df

它会使用 CSV 生成一个 DataFrame，但是认为第一行包含字段名称。我们需要向该函数的参数 `names` 提供一个字段名称列表。

In [ ]:
price_df = pd.read_csv('prices.csv', names=['ticker', 'date', 'open', 'high', 'low',
                                             'close', 'volume', 'adj_close', 'adj_volume'])

price_df

## DataFrame 计算
将数据放入 DataFrame 后，我们可以对其进行运算。我们使用 [`DataFrame.median`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.median.html) 函数算出每支股票的中位数。

In [ ]:
price_df.median()

不正确。这些值是整个股票备选投资组合的中位数。我们将使用 [`DataFrame.groupby`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.groupby.html) 函数获取每支股票的均值。

In [ ]:
price_df.groupby('ticker').median()

这就是我们要计算的值。但是，我们不希望每次进行运算都运行 `groupby` 函数。我们可通过 `price_df_ticker_groups = price_df.groupby('ticker')` 保存该 GroupBy 对象。这限制我们只能对 GroupBy 对象进行运算。还可以使用 [`GroupBy.apply`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.core.groupby.GroupBy.apply.html)，但是会降低性能。真正的问题是数据的表示形式。

In [ ]:
price_df.iloc[:16]

你能发现问题吗？花点时间找找问题。

问题出在行 [6, 7] 和 [13, 14] 之间

In [ ]:
price_df.iloc[[6, 7, 13, 14]]

所有股票的数据堆叠到一起了。我们用二维空间表示三维数据。我们使用 Panda 的 Panel 解决了此问题，Panel [已被弃用](https://pandas.pydata.org/pandas-docs/version/0.21.0/dsintro.html#deprecate-panel)。Pandas 文档建议我们使用 [MultiIndex](https://pandas.pydata.org/pandas-docs/version/0.21/advanced.html) 或[xarray](http://xarray.pydata.org/en/stable/)。[MultiIndex](https://pandas.pydata.org/pandas-docs/version/0.21/advanced.html) 依然不能解决我们的问题，因为数据依然用二维空间表示。[xarray](http://xarray.pydata.org/en/stable/) 能够存储三维数据，但是 Finance 使用的是 Pandas，所以我们将继续使用此库。学完此课程后，建议看看 [xarray](http://xarray.pydata.org/en/stable/)。

如何在 Pandas 中使用三维数据？我们可以将每个第三维拆分成它自己的二维 DataFrame。以下面的数组为例：
```
[
    [
        [ 0,  1],
        [ 2,  3],
        [ 4,  5]
    ],[
        [ 6,  7],
        [ 8,  9],
        [10, 11]
    ],[
        [12, 13],
        [14, 15],
        [16, 17]
    ],[
        [18, 19],
        [20, 21],
        [22, 23]
    ]
]    
```

我们需要将它拆分成以下两个二维数组：
```
[
    [0, 2, 4],
    [6, 8, 10],
    [12, 14, 16],
    [18, 20, 22]
] 
[
    [1, 3, 5],
    [7, 8, 11],
    [13, 15, 17],
    [19, 21, 23]
] 
```

对于此示例来说，第三维是“open”、“high”、“low”、“close”、“volume”、“adj_close”和“adj_volume”。我们将使用 [`DataFrame.pivot`](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.pivot.html) 函数生成这些 DataFrame。

In [ ]:
open_prices = price_df.pivot(index='date', columns='ticker', values='open')
high_prices = price_df.pivot(index='date', columns='ticker', values='high')
low_prices = price_df.pivot(index='date', columns='ticker', values='low')
close_prices = price_df.pivot(index='date', columns='ticker', values='close')
volume = price_df.pivot(index='date', columns='ticker', values='volume')
adj_close_prices = price_df.pivot(index='date', columns='ticker', values='adj_close')
adj_volume = price_df.pivot(index='date', columns='ticker', values='adj_volume')

open_prices

这样就可以获得所有开盘价、最高价、最低价等的 DataFrame。我们最终要计算的是每支股票的均值。

In [ ]:
open_prices.mean()

还可以通过转置获得每个日期的均值。

In [ ]:
open_prices.T.mean()

日期在索引中、股票代号在列中或反过来都没关系。始终相互转置。由于我们将对日期进行很多运算，所以在这门课程中，我们将把日期放入索引中，并把股票代号放入列中。
## 小测验
下面看看你能否运用所学的知识。请实现 `csv_to_close` 函数，它将接受文件路径和 `csv_filename`，并输出收盘价二维数组。你可以认为 `csv_to_close` 使用的 CSV 文件具有和“prices.csv”相同的名称并且字段顺序一样。

为了帮助你完成小测验，我们提供了单元测试供你测试函数实现情况。对于这道练习，我们将使用 `quiz_tests` 模块中的函数 `test_csv_to_close` 测试 `csv_to_close`。

In [ ]:
import quiz_tests


def csv_to_close(csv_filepath, field_names):
    """Reads in data from a csv file and produces a DataFrame with close data.
    
    Parameters
    ----------
    csv_filepath : str
        The name of the csv file to read
    field_names : list of str
        The field names of the field in the csv file

    Returns
    -------
    close : DataFrame
        Close prices for each ticker and date
    """
    
    # TODO: Implement Function
    
    return None


quiz_tests.test_csv_to_close(csv_to_close)

## 小测验解答
如果你遇到问题，请在[此处](stock_data_solution.ipynb)查看解答。